In [1]:
import requests
import re
import json
import pandas as pd
from pprint import pprint
from time import sleep
import os

from spotify_credentials import client_id, client_secret

In [ ]:
#getting spotify access token
url = "https://accounts.spotify.com/api/token"
header = {"Content-Type": "application/x-www-form-urlencoded"}
data = {"grant_type":"client_credentials",
        "client_id" : client_id,
        "client_secret" : client_secret}

r = requests.post(url, data=data, headers=header).json()

spotify_api_token = r['access_token']

In [1]:
#reading in a csv with rows containing song and artist details. Converting this csv into a dataframe.
songdf = pd.read_csv(r"C:\Users\kevin\bootcamp\Project_1\resources\all_top100songs_by_year.csv")
songdf["id"] = ""
#removing unnecessary index columns
songdf = songdf.iloc[:,[2,3,4,5,6]]

NameError: name 'pd' is not defined

In [ ]:
#This block of code iterates over the dataframe songdf set above. At each row of the DataFrame# 
#it gets artist and song values and formats them, then passes these values into the Spotify search endpoint (https://developer.spotify.com/documentation/web-api/reference/search), which returns a unique song ID.
#This song id value is assigned to the [row, id] position, for the current row.
#a csv with song IDs added to each row is created.

#setting Spotify endpoint header
spotify_headers = {'Authorization': 'Bearer {}'.format(spotify_api_token)}

for index, row in songdf.iterrows():#https://www.geeksforgeeks.org/iterating-over-rows-and-columns-in-pandas-dataframe/
  
    #https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.at.html
    artist = re.split(' Featuring | With | Duet | X | x | & |, | \\(| And | Feat.', songdf.iat[index,1])[0].split()
    artist_final = '+'.join(artist)

    song = re.split(' \\(', songdf.iat[index,0])[0].replace("/"," ").split()
    song_final = '+'.join(song)
    

    try:
        search_url = f"https://api.spotify.com/v1/search?q=track%3A{song_final}+artist%3A{artist_final}&type=track"
        search_response = requests.get(search_url, headers = spotify_headers).json()
        song_id = search_response["tracks"]["items"][0]["id"]
                
    except:
        print(f"no ID available for song named: {song_final} by {artist_final}")
        song_id = ""
        pass
    
    songdf.iat[index,4] = str(song_id)
   

print("---------------")
print("calls complete.")
print("---------------")


path = r"C:\Users\kevin\bootcamp\Project_1\resources"
songdf.to_csv(os.path.join(path,f"top100_allYears_withID.csv")) #https://stackoverflow.com/questions/22872952/set-file-path-for-to-csv-in-pandas
